# This is a custom class that implements the RSA encryption protocol

In [14]:
#We will mostly develop our own methods
#However we will use these modules
import numpy as np
import math
import string

In [15]:
"""Create a custom hash functions
 from strings to lists of integers."""

import numpy as np
import string


"""General Hash class"""

class Hash_class:
    hash_table={}
    dehash_table={}

"""Elementary hash class:
   Each letter is individually hashed consecutively from A:2 to Z:27
"""
class Simple_hash(Hash_class):
    
    
    def __init__(self,k=2):
        #Create the alphabet+digits
        alph=list(string.ascii_uppercase)
        digits=[str(i) for i in range(10)]
        alph=alph+digits
        #Create the hash values
        hash_values=list(i for i in range(k,k+36))
        #Create the hash dictionary
        self.hash_table=dict(zip(alph,hash_values))
        self.dehash_table=dict(zip(hash_values,alph))
        
    
    #Hash the plaintext
    def hash_it(self,plaintext):
        split_plaintext=self.split_it(plaintext)
        hashed_plaintext=np.empty(0,dtype=int)
        for i in split_plaintext:
           hashed_plaintext=np.append(hashed_plaintext,self.hash_table[i])
        return hashed_plaintext
    
    #Clean and transform the plaintext into a list of pairs of letters
    def split_it(self,plaintext):
        bad_char=[" ",",",".","'","!","?",":",";","\n",'"'
                  "*", "&", "^", "%", "$", "#", "@", "~","/"]
        
        for i in bad_char:
            plaintext=plaintext.replace(i,"")
        plaintext=plaintext.upper()
        return plaintext



"""Hash class for hashing pairs 'AA', 'AB',..., 'ZZ' """
class Biletter_hash(Hash_class):
    
    
    def __init__(self,N=100000):
        #Create the bi-alphabet, including combinations with digits
        alph=list(string.ascii_uppercase)
        digits=[str(i) for i in range(10)]
        alph=alph+digits
        dialph=list([])
        for i in alph:
            for j in alph:
                x=i+j
                dialph.append(x)
                
        #Create the random hash values
        hash_values={}
        while len(hash_values)<len(dialph):
            r=np.random.randint(N)
            if r not in hash_values:
                hash_values[r]=True
        hash_values=list(hash_values.keys())

        #Create the hash dictionary
        self.hash_table=dict(zip(dialph,hash_values))
        
        #Create a trivial dehash table for un-encrypted hashed text
        self.dehash_table=dict(zip(hash_values,dialph))
        
        
    #Hash the plaintext
    def hash_it(self,plaintext):
        split_plaintext=self.split_it(plaintext)
        hashed_plaintext=np.empty(0,dtype=int)
        for i in split_plaintext:
           hashed_plaintext=np.append(hashed_plaintext,self.hash_table[i])
        return hashed_plaintext
    
    
    #Clean and transform the plaintext into a list of pairs of letters   
    def split_it(self,plaintext):
        bad_char=[" ",",",".","'","!","?",":",";","\n",'"'
                  "*", "&", "^", "%", "$", "#", "@", "~","/"]
        
        for i in bad_char:
            plaintext=plaintext.replace(i,"")
        plaintext=plaintext.upper()
        if len(plaintext)%2!=0: #Make the length of the text even
            plaintext+="X"
        i=0
        split_plaintext=list([])
        for  i in range(0,len(plaintext),2):
            x=plaintext[i:i+2]
            split_plaintext.append(x)
        
        return split_plaintext

In [16]:
#Example of hashing a simple message in two ways
#Single letter simple hash
pl="Carpe Diem!"
f=Simple_hash()
a=f.split_it(pl)
b=f.hash_it(pl)
print(a)
print(b)

#Bi-letter advanced hash
pl="Carpe Diem!"
f=Biletter_hash()
a=f.split_it(pl)
b=f.hash_it(pl)
print(a)
print(b)

CARPEDIEM
[ 4  2 19 17  6  5 10  6 14]
['CA', 'RP', 'ED', 'IE', 'MX']
[31082 25116  7253 88003 70117]


In [17]:
#Define a few usefull functions

#Fast exponentiation modulo an integer using consecutive squaring
#compute a**x mod n
def pow_mod(a,x,n):
    result=1
    temp_a, temp_x = a, x#if we do not wish to modify a or x
    while temp_x>0:
        if temp_x%2!=0:
            result=(result*temp_a) % n
        temp_a=temp_a**2 % n   
        temp_x=temp_x//2
    return result

#Finding the Greatest Common Divisor
def gcd(a=1,b=1):
    if a==0:
        return b
    g=gcd(b%a,a)
    return g
    
    
#Extended Euclidean Algorithm: provides both the greatest common divisor and the inverse modulo the other number
def generalized_Euclidean(a=1,b=1):    
    if a == 0:
        return (b, 0, 1)
    else:
        (gcd, x, y) = generalized_Euclidean(b%a, a)
        return (gcd, (y - int(b / a)*x), x)



In [18]:
#Implementing the RSA protocol as a custom class
class RSA():
    """
    Atributes:
    Prime numbers p and q
    modulus=pq
    phi=(p-1)*(q-1)
    public_key 
    private_key
    plaintext
    cryptext
    hash_method
    hash_table
    dehash_table
    """
    def __init__(self,p=2,q=3,public_key=1,hash_method=Simple_hash(2)):
            self.p=p
            self.q=q
            self.modulus=p*q
            self.phi=(p-1)*(q-1)
            self.plaintext=str()
            self.cryptext=[]
            #The public key must be coprime with phi
            while gcd(public_key,self.phi)>1:
                print("The public key is invalid, not invertible modulo n\n")
                public_key=int(input("New public key: "))
            self.public_key=public_key
            #Calculating the private key from the generalized Euclidean algorithm
            t=generalized_Euclidean(self.public_key,self.phi)
            self.private_key=(t[1]+self.phi)%self.phi
            self.hash_method=hash_method
            self.hash_table=hash_method.hash_table
            #Precompute the dehash table
            dehash_table={}
            for key,value in self.hash_table.items():
                temp=self.encrypt_Block(value)
                dehash_table[temp]=key
            self.dehash_table=dehash_table
    
    
    """Function to encrypt a single block of plain text"""
    def encrypt_Block(self,hashed):
        return pow_mod(hashed, self.public_key, self.modulus)
    
    
    """Function to decrypt a single block"""
    def decrypt_Block(self,hashed):
        return pow_mod(hashed, self.private_key, self.modulus)
    
    
    """Encrypt a text message"""
    def encrypt_text(self,plaintext):
        
        
        self.plaintext=self.hash_method.hash_it(plaintext)
        
        self.cryptext=np.empty(0,dtype=int)
        for x in self.plaintext:
            y=self.encrypt_Block(int(x))
            self.cryptext=np.append(self.cryptext,y)
    
    
    #Decrypt a cryptext from a list of hash values
    def decrypt_message(self, cryptext):
            
            self.decrypted=""
            
            cryptextlist=cryptext.tolist()
            
            for y in cryptextlist:
                self.decrypted+=str(self.dehash_table[y])

In [19]:
#Examples of using the RSA protocol with simple hash
plaintext="Lord, what fools these mortals be!\n A Midsummer Night's Dream \n William Shakespeare"

prot=RSA(31,43,11)
prot.encrypt_text(plaintext)
print(prot.cryptext)


[ 840   47   72  335 1003  169  715  508 1198   47   47  840  534  508
  169 1111  534 1111  784   47   72  508  715  840  534 1191 1111  715
  784  918  335  534 1040  784  784 1111   72  418  918 1279  169  508
  534  335   72 1111  715  784 1003  918  840  840  918  715  784  534
  169  715 1230 1111  534  239 1111  715   72 1111]


In [20]:
#Decrypt the previous message
prot.decrypt_message(prot.cryptext)
print(prot.decrypted)

LORDWHATFOOLSTHESEMORTALSBEAMIDSUMMERNIGHTSDREAMWILLIAMSHAKESPEARE


In [21]:
#Example of using the RSA protocol with advanced random biletter hash
plaintext="Lord, what fools these mortals be!\n A Midsummer Night's Dream \n William Shakespeare"

prot=RSA(1228,997,public_key=11,hash_method=Biletter_hash())
prot.encrypt_text(plaintext)
print(prot.cryptext)

[  90119  870912  516555  184952  173395  457629  133076  938548 1025365
  845664  879449   98036  171140 1204243   91227  652376  449372 1095552
   79319  135772  634424   84257 1099420  189228  208143  576120 1149336
  805995 1076687  870904  659772 1204243 1099420]


In [22]:
#Decrypt the previous message
prot.decrypt_message(prot.cryptext)
print(prot.decrypted)

LORDWHATFOOLSTHESEMORTALSBEAMIDSUMMERNIGHTSDREAMWILLIAMSHAKESPEARE


In [23]:
#A second example of using the RSA protocol with advanced random biletter hash
plaintext="Lord, what fools these mortals be!\n A Midsummer Night's Dream \n William Shakespeare"

prot=RSA(460589,862067,public_key=11,hash_method=Biletter_hash())
prot.encrypt_text(plaintext)
print(prot.cryptext)

[266980283651 208279191477 288581538244  66826432387 149822758302
 339427930936  92177107515 314577568067 304841940335 128368115502
 147634091105 163929769868 196547365730 173751688222 139190587717
  56728758238 277269257395 149722128503 204710645960 285220479845
 243065760630 380727351091 271367353496 369661464007  21875722787
 134097651372 151959747170 324657169233 339969278683 180589214687
 369604960696 173751688222 271367353496]


In [24]:
#Decrypt the previous message
prot.decrypt_message(prot.cryptext)
print(prot.decrypted)

LORDWHATFOOLSTHESEMORTALSBEAMIDSUMMERNIGHTSDREAMWILLIAMSHAKESPEARE


In [25]:
#The private key might be very large compared to the public key
prot.private_key

72192228147